---
## 04b_Scoring_Yearly
---

# Notebook 04b: Composite Scoring (Extended)

This notebook extends the composite scoring analysis with additional breakdowns and checks.  

## Imports and Settings
Import libraries for scoring, visualization, and scaling.  
Configure display settings for readability.  

In [ ]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

## Directory Setup and File Check
Define directories for normalized, scored, and composite datasets.  
Confirm availability of normalized input files before running extended scoring analysis.  

In [ ]:
base_dir      = "../data"
norm_dir      = os.path.join(base_dir, "normalized")
score_dir     = os.path.join(base_dir, "scored")
composite_dir = os.path.join(base_dir, "composite")

print("Normalized datasets available:")
for f in os.listdir(norm_dir):
    print(" -", f)

Normalized datasets available:
 - electricity_normalized.csv
 - gdp_ppp_normalized.csv
 - gov_effect_normalized.csv
 - internet_normalized.csv
 - literacy_normalized.csv
 - mobile_normalized.csv
 - researchers_normalized.csv
 - rnd_gdp_normalized.csv
 - tertiary_normalized.csv


## Load Normalized Datasets
Load all normalized indicator files from `/data/normalized/` into a dictionary of DataFrames.  
Preview one dataset to confirm successful loading and structure.  

In [ ]:

norm_dfs = {}
for f in os.listdir(norm_dir):
    if f.endswith("_normalized.csv"):
        key = f.replace("_normalized.csv", "")
        norm_dfs[key] = pd.read_csv(os.path.join(norm_dir, f))

# Quick preview one dataset
for k, v in norm_dfs.items():
    print(k, v.shape)
    display(v.head(3))
    break

electricity (9310, 6)


,Country Name,Country Code,Year,Indicator,Value,Normalized
0,Aruba,ABW,1990.0,electricity,100.0,100.0
1,Africa Eastern and Southern,AFE,1990.0,electricity,NaN,NaN
2,Afghanistan,AFG,1990.0,electricity,NaN,NaN


## Reshape Datasets into Wide Format
Convert each normalized dataset into wide format:  
- Retain `Country Name`, `Country Code`, and `Year`.  
- Rename the `Value` column to the indicator’s name.  
- Store reshaped datasets for later merging.  

Preview one reshaped dataset to confirm structure.  

In [4]:
# Re-shape each dataset: wide format with indicator column renamed
reshaped_dfs = []

for key, df in norm_dfs.items():
    temp = df[["Country Name", "Country Code", "Year", "Value"]].copy()
    temp = temp.rename(columns={"Value": key})
    reshaped_dfs.append(temp)

# Quick check on one reshaped dataset
reshaped_dfs[0].head()

,Country Name,Country Code,Year,electricity
0,Aruba,ABW,1990.0,100.0
1,Africa Eastern and Southern,AFE,1990.0,NaN
2,Afghanistan,AFG,1990.0,NaN
3,Africa Western and Central,AFW,1990.0,NaN
4,Angola,AGO,1990.0,NaN


## Merge All Reshaped Datasets
Use `functools.reduce` to merge the reshaped indicator datasets into a single panel:  
- Keys: `Country Name`, `Country Code`, and `Year`.  
- Join type: outer (preserves all available data across indicators).  

Display dataset shape and preview the first few rows.  

In [ ]:
from functools import reduce

merged_df = reduce(
    lambda left, right: pd.merge(left, right, on=["Country Name", "Country Code", "Year"], how="outer"),
    reshaped_dfs
)

print("Merged panel dataset shape:", merged_df.shape)
display(merged_df.head())

Merged panel dataset shape: (9560, 12)


,Country Name,Country Code,Year,electricity,gdp_ppp,gov_effect,internet,literacy,mobile,researchers,rnd_gdp,tertiary
0,Afghanistan,AFG,1990.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,2.46528
1,Afghanistan,AFG,1991.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
2,Afghanistan,AFG,1992.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,Afghanistan,AFG,1993.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
4,Afghanistan,AFG,1994.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


## Compute Equal-Weight and Weighted Scores
Steps:
1. Identify indicator columns (exclude identifiers and score placeholders).  
2. Forward-fill missing values within each country’s time series.  
3. Re-scale indicators to a 0–100 range as a safeguard.  
4. Compute **Equal-Weight Score**: mean of all indicators.  
5. Compute **Weighted Score**: weighted average using predefined weights.  

This produces a panel dataset with yearly scores for all available countries.  

In [ ]:
from sklearn.preprocessing import MinMaxScaler

indicator_cols = [col for col in merged_df.columns if col not in ["Country Name", "Country Code", "Year", "Equal_Score", "Weighted_Score"]]

# Forward-fill missing values per country
merged_df = merged_df.sort_values(["Country Name", "Year"])
for col in indicator_cols:
    merged_df[col] = merged_df.groupby("Country Name")[col].ffill()

# Re-scale indicators to 0–100 (safeguard)
scaler = MinMaxScaler((0,100))
merged_df[indicator_cols] = scaler.fit_transform(merged_df[indicator_cols])

# Equal-weight score
merged_df["Equal_Score"] = merged_df[indicator_cols].mean(axis=1, skipna=True)

# Weighted score
weights = {
    "gov_effect": 0.25,
    "gdp_ppp": 0.20,
    "literacy": 0.20,
    "internet": 0.10,
    "mobile": 0.10,
    "electricity": 0.05,
    "tertiary": 0.05,
    "researchers": 0.025,
    "rnd_gdp": 0.025
}

# Normalize to sum = 1
total_weight = sum(weights.values())
weights = {k: v/total_weight for k,v in weights.items()}

def weighted_average(row, cols, weights):
    vals, wts = [], []
    for col in cols:
        if col in weights and pd.notnull(row[col]):
            vals.append(row[col])
            wts.append(weights[col])
    if len(vals) == 0:
        return np.nan
    return np.average(vals, weights=wts)

merged_df["Weighted_Score"] = merged_df.apply(
    lambda row: weighted_average(row, indicator_cols, weights), axis=1
)

print("Panel dataset with yearly scores:", merged_df.shape)
display(merged_df.head(10))

Panel dataset with yearly scores: (9560, 14)


,Country Name,Country Code,Year,electricity,gdp_ppp,gov_effect,internet,literacy,mobile,researchers,rnd_gdp,tertiary,Equal_Score,Weighted_Score
0,Afghanistan,AFG,1990.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.479177,0.493059,0.295835
1,Afghanistan,AFG,1991.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.479177,0.493059,0.295835
2,Afghanistan,AFG,1992.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.479177,0.493059,0.295835
3,Afghanistan,AFG,1993.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.479177,0.493059,0.295835
4,Afghanistan,AFG,1994.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.479177,0.493059,0.295835
5,Afghanistan,AFG,1995.0,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,1.479177,0.493059,0.295835
6,Afghanistan,AFG,1996.0,NaN,NaN,5.398536,0.0,NaN,0.0,NaN,NaN,1.479177,1.719428,2.847186
7,Afghanistan,AFG,1997.0,NaN,NaN,5.398536,0.0,NaN,0.0,NaN,NaN,1.479177,1.719428,2.847186
8,Afghanistan,AFG,1998.0,NaN,NaN,6.882781,0.0,NaN,0.0,NaN,NaN,1.479177,2.090490,3.589308
9,Afghanistan,AFG,1999.0,NaN,NaN,6.882781,0.0,NaN,0.0,NaN,NaN,1.479177,2.090490,3.589308


## Save Yearly Scores
Export yearly composite scores for all countries:  
- **Equal-Weight Score** → `/data/scored/yearly_equal_scores.csv`  
- **Weighted Score** → `/data/scored/yearly_weighted_scores.csv`  

These files preserve time-series results for downstream trajectory and clustering analysis.  

In [ ]:
equal_out = merged_df[["Country Name", "Country Code", "Year", "Equal_Score"]]
equal_out.to_csv(os.path.join(score_dir, "yearly_equal_scores.csv"), index=False)

weighted_out = merged_df[["Country Name", "Country Code", "Year", "Weighted_Score"]]
weighted_out.to_csv(os.path.join(score_dir, "yearly_weighted_scores.csv"), index=False)

print("Yearly scores saved:")
print(" -", os.path.join(score_dir, "yearly_equal_scores.csv"))
print(" -", os.path.join(score_dir, "yearly_weighted_scores.csv"))

Yearly scores saved:
 - ../data\scored\yearly_equal_scores.csv
 - ../data\scored\yearly_weighted_scores.csv


## Save Full Panel Dataset
Export the complete panel dataset (all indicators, yearly scores) for use in **Notebook 06: Clustering Trajectories**.  
This dataset provides the foundation for trajectory analysis and cluster identification.  

In [ ]:
panel_out = os.path.join(score_dir, "panel_dataset.csv")
merged_df.to_csv(panel_out, index=False)

print(f"Panel dataset saved to {panel_out} with shape {merged_df.shape}")
print(merged_df.head())

✔ Panel dataset saved to ../data\scored\panel_dataset.csv with shape (9560, 14)
  Country Name Country Code    Year  electricity  gdp_ppp  gov_effect  internet  literacy  mobile  researchers  rnd_gdp  tertiary  Equal_Score  Weighted_Score
0  Afghanistan          AFG  1990.0          NaN      NaN         NaN       0.0       NaN     0.0          NaN      NaN  1.479177     0.493059        0.295835
1  Afghanistan          AFG  1991.0          NaN      NaN         NaN       0.0       NaN     0.0          NaN      NaN  1.479177     0.493059        0.295835
2  Afghanistan          AFG  1992.0          NaN      NaN         NaN       0.0       NaN     0.0          NaN      NaN  1.479177     0.493059        0.295835
3  Afghanistan          AFG  1993.0          NaN      NaN         NaN       0.0       NaN     0.0          NaN      NaN  1.479177     0.493059        0.295835
4  Afghanistan          AFG  1994.0          NaN      NaN         NaN       0.0       NaN     0.0          NaN      NaN  1.47

# Wrap-Up: Yearly Scoring and Panel Dataset

## Notebook 04b Wrap-Up

### What We Did
1. **Loaded all normalized datasets**  
   - Electricity, GDP PPP, Governance Effectiveness, Internet, Literacy, Mobile, Researchers, R&D (% GDP), Tertiary Education.  

2. **Reshaped datasets**  
   - Converted each file into a common structure:  
     `Country Name | Country Code | Year | <indicator>`  

3. **Merged into a panel dataset**  
   - Combined all indicators by `Country Name + Country Code + Year`.  
   - Result: ~9,500 country-year rows with 14 columns.  

4. **Calculated yearly composite scores**  
   - Equal-weight score: mean of available indicators per row.  
   - Weighted score: governance (0.25), GDP PPP (0.20), literacy (0.20), internet (0.10), mobile (0.10), electricity (0.05), tertiary (0.05), researchers (0.025), R&D (0.025).  
   - Weights normalized to sum = 1.  

5. **Saved outputs**  
   - `/data/scored/yearly_equal_scores.csv`  
   - `/data/scored/yearly_weighted_scores.csv`  
   - `/data/scored/panel_dataset.csv`  

---

### Key Takeaways
- **Notebook 04 (snapshot)** shows *who is most ready now*.  
- **Notebook 04b (yearly panel)** shows *how readiness evolves over time*.  
- This dataset is the foundation for:  
  - Growth trajectory visuals (**Notebook 05**)  
  - Clustering and trajectory modeling (**Notebook 06**)  